In [ ]:
from wavelet1d import *
from scipy.integrate import quad
from scipy.linalg import eigvalsh

# Primal MRA

In [ ]:
d = 3
mra = PrimalMRA(d)

In [ ]:
print(mra.ML)

ML_test = PrimalMRA(2).ML
ML_ref = np.array([[1],[1/2]])
assert np.allclose(ML_test, ML_ref)

ML_test = PrimalMRA(3).ML
ML_ref = np.array([[1, 0], [1/2, 1/2], [0, 3/4], [0, 1/4]])
assert np.allclose(ML_test, ML_ref)

- Primbs - 2006, Beispiel 3.18

In [ ]:
j = 5
M0 = mra.refinement_matrix(j)
plt.rcParams['figure.figsize'] = [5, 10]
plt.spy(M0, marker='o', markersize=5)
plt.show()

- Primbs - 2006, Abbildung 3.5

In [ ]:
j = 3
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
mra.plot(j)

# basis functions constructed using refinement relation
mra.plot(j, nu=0, from_refine_mat=True)

- Primbs - 2006, Abbildung 3.2-3.4

In [ ]:
j = 4
G = mra.gramian(j)

bs = mra.basis_functions(j)
n = G.shape[0]
G_quad = np.empty((n, n))
for k in range(n):
    supp_k = mra.support(j, k)
    for m in range(n):
        supp_m = mra.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            G_quad[k, m] = 0.
        else:
            G_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(G, G_quad)

In [ ]:
G_test = PrimalMRA(3).gramian(3)
G_ref = np.array([
    [1/5, 7/60, 1/60, 0, 0, 0, 0, 0, 0, 0],
    [7/60, 1/3, 5/24, 1/120, 0, 0, 0, 0, 0, 0],
    [1/60, 5/24, 11/20, 13/60, 1/120, 0, 0, 0, 0, 0],
    [0, 1/120, 13/60, 11/20, 13/60, 1/120, 0, 0, 0, 0],
    [0, 0, 1/120, 13/60, 11/20, 13/60, 1/120, 0, 0, 0],
    [0, 0, 0, 1/120, 13/60, 11/20, 13/60, 1/120, 0, 0],
    [0, 0, 0, 0, 1/120, 13/60, 11/20, 13/60, 1/120, 0],
    [0, 0, 0, 0, 0, 1/120, 13/60, 11/20, 5/24, 1/60],
    [0, 0, 0, 0, 0, 0, 1/120, 5/24, 1/3, 7/60],
    [0, 0, 0, 0, 0, 0, 0, 1/60, 7/60, 1/5]])
assert np.allclose(G_test, G_ref)

- Primbs - 2008, Section 5.1

In [ ]:
# riesz constants
n = G.shape[0]
c1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
c2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("c1 =", c1)
print("c2 =", c2)
print("c2/c1 =", c2/c1)

tests = [(2, 5), (3, 5), (4, 6)]
results = []
refs = [[0.5, 0.999], [0.308, 0.999], [0.193, 0.999]]
for d_test, j_test in tests:
    G_test = PrimalMRA(d_test).gramian(j_test)
    n = G_test.shape[0]
    c1_test = np.sqrt(eigvalsh(G_test, subset_by_index=[0, 0])[0])
    c2_test = np.sqrt(eigvalsh(G_test, subset_by_index=[n-1, n-1])[0])
    results.append([c1_test, c2_test])
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2010, Table 1

In [ ]:
# diagonal scaling
n = G.shape[0]
D = np.diag(1 / np.sqrt(np.diag(G)))
G = D @ G @ D
c1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
c2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("c1 =", c1)
print("c2 =", c2)
print("c2/c1 =", c2/c1)

tests = [(2, 5), (3, 5), (4, 6)]
results = []
refs = [[0.707, 1.225], [0.489, 1.352], [0.329, 1.453]]
for (d_test, j_test) in tests:
    G_test = PrimalMRA(d_test).gramian(j_test)
    D = np.diag(1 / np.sqrt(np.diag(G_test)))
    G_test = D @ G_test @ D
    n = G_test.shape[0]
    c1_test = np.sqrt(eigvalsh(G_test, subset_by_index=[0, 0])[0])
    c2_test = np.sqrt(eigvalsh(G_test, subset_by_index=[n-1, n-1])[0])
    results.append([c1_test, c2_test])
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2006, Tabelle 6.3

In [ ]:
j = 4
nu = 1
A = mra.inner_product(j, nu)

bs = mra.basis_functions(j, nu)
n = A.shape[0]
A_quad = np.empty((n, n))
for k in range(n):
    supp_k = mra.support(j, k)
    for m in range(n):
        supp_m = mra.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            A_quad[k, m] = 0.
        else:
            A_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(A, A_quad)

In [ ]:
A_test = PrimalMRA(3).inner_product(3, nu=1)
A_ref = np.array([
    [256/3, -64, -64/3, 0, 0, 0, 0, 0, 0, 0],
    [-64, 256/3, -32/3, -32/3, 0, 0, 0, 0, 0, 0],
    [-64/3, -32/3, 64, -64/3, -32/3, 0, 0, 0, 0, 0],
    [0, -32/3, -64/3, 64, -64/3, -32/3, 0, 0, 0, 0],
    [0, 0, -32/3, -64/3, 64, -64/3, -32/3, 0, 0, 0],
    [0, 0, 0, -32/3, -64/3, 64, -64/3, -32/3, 0, 0],
    [0, 0, 0, 0, -32/3, -64/3, 64, -64/3, -32/3, 0],
    [0, 0, 0, 0, 0, -32/3, -64/3, 64, -32/3, -64/3],
    [0, 0, 0, 0, 0, 0, -32/3, -32/3, 256/3, -64],
    [0, 0, 0, 0, 0, 0, 0, -64/3, -64, 256/3]])
assert np.allclose(A_test, A_ref)

- Primbs - 2006, Beispiel 3.26

# Primal MRA & Boundary Conditions

In [ ]:
d = 3
mra = PrimalMRA(d, bc=True)

In [ ]:
print(mra.ML)

In [ ]:
j = 5
M0 = mra.refinement_matrix(j)
plt.rcParams['figure.figsize'] = [5, 10]
plt.spy(M0, marker='o', markersize=5)
plt.show()

In [ ]:
j = 3
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
mra.plot(j)

# basis functions constructed using refinement relation
mra.plot(j, nu=0, from_refine_mat=True)

In [ ]:
j = 4
G = mra.gramian(j)

bs = mra.basis_functions(j)
n = G.shape[0]
G_quad = np.empty((n, n))
for k in range(n):
    supp_k = mra.support(j, k)
    for m in range(n):
        supp_m = mra.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            G_quad[k, m] = 0.
        else:
            G_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(G, G_quad)

In [ ]:
# riesz constants
n = G.shape[0]
c1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
c2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("c1 =", c1)
print("c2 =", c2)
print("c2/c1 =", c2/c1)

tests = [(2, 5), (3, 5), (4, 6)]
results = []
refs = [[0.579, 0.999], [0.365, 0.999], [0.221, 0.999]]
for d_test, j_test in tests:
    G_test = PrimalMRA(d_test, bc=True).gramian(j_test)
    n = G_test.shape[0]
    c1_test = np.sqrt(eigvalsh(G_test, subset_by_index=[0, 0])[0])
    c2_test = np.sqrt(eigvalsh(G_test, subset_by_index=[n-1, n-1])[0])
    results.append([c1_test, c2_test])
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2006, Tabelle 6.4

In [ ]:
# diagonal scaling
n = G.shape[0]
D = np.diag(1 / np.sqrt(np.diag(G)))
G = D @ G @ D
c1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
c2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("c1 =", c1)
print("c2 =", c2)
print("c2/c1 =", c2/c1)

tests = [(2, 5), (3, 5), (4, 6)]
results = []
refs = [[0.709, 1.224], [0.495, 1.347], [0.336, 1.445]]
for (d_test, j_test) in tests:
    G_test = PrimalMRA(d_test, bc=True).gramian(j_test)
    D = np.diag(1 / np.sqrt(np.diag(G_test)))
    G_test = D @ G_test @ D
    n = G_test.shape[0]
    c1_test = np.sqrt(eigvalsh(G_test, subset_by_index=[0, 0])[0])
    c2_test = np.sqrt(eigvalsh(G_test, subset_by_index=[n-1, n-1])[0])
    results.append([c1_test, c2_test])
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2006, Tabelle 6.5

In [ ]:
j = 4
nu = 1
A = mra.inner_product(j, nu)

bs = mra.basis_functions(j, nu)
n = A.shape[0]
A_quad = np.empty((n, n))
for k in range(n):
    supp_k = mra.support(j, k)
    for m in range(n):
        supp_m = mra.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            A_quad[k, m] = 0.
        else:
            A_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(A, A_quad)

# Dual MRA

In [ ]:
d = 3
d_t = 5
mra = PrimalMRA(d)
mra_t = DualMRA(d, d_t)

In [ ]:
ML_test = DualMRA(3, 5).ML
ML_ref = np.array([
    [5/3, -2359/3840, 101909/345600, -17611/115200, 3119/57600, -61/6912],
    [2/3, 2359/1920, -101909/172800, 17611/57600, -3119/28800, 61/3456],
    [-1/2, 2503/1280, -18413/115200, 1027/38400, -83/19200, 1/2304],
    [1/6, -1243/3840, 573353/345600, -79687/115200, 13223/57600, -253/6912],
    [0, -77/256, 2583/2560, 789/2560, -181/1280, 19/768],
    [0, -21/256, 399/2560, 2877/2560, -333/1280, 9/256],
    [0, 105/512, -547/1024, 1523/1024, -79/512, 43/512],
    [0, -35/512, 129/1024, -145/1024, 709/512, -587/1536],
    [0, 0, 15/256, -97/256, 175/128, -13/128],
    [0, 0, -5/256, 19/256, -13/128, 175/128],
    [0, 0, 0, 15/256, -97/256, 175/128],
    [0, 0, 0, -5/256, 19/256, -13/128],
    [0, 0, 0, 0, 15/256, -97/256],
    [0, 0, 0, 0, -5/256, 19/256],
    [0, 0, 0, 0, 0, 15/256],
    [0, 0, 0, 0, 0, -5/256]])
assert np.allclose(ML_test, ML_ref)

- Primbs - 2006, Beispiel 4.20

In [ ]:
j = 5
M0 = mra_t.refinement_matrix(j)
plt.rcParams['figure.figsize'] = [5, 10]
plt.spy(M0, marker='o', markersize=5)
plt.show()

- Primbs - 2006, Abbildung 4.5

In [ ]:
# biorthogonality
j = 5
M0 = mra.refinement_matrix(j)
M0_t = mra_t.refinement_matrix(j)
assert np.allclose(M0.T @ M0_t, np.identity(M0.shape[1]))

# Dual MRA & Boundary Conditions & d = 2

In [ ]:
d = 2
d_t = 4
mra = PrimalMRA(d, bc=True)
mra_t = DualMRA(d, d_t, bc=True)

In [ ]:
j = 5
M0 = mra_t.refinement_matrix(j)
plt.rcParams['figure.figsize'] = [5, 10]
plt.spy(M0, marker='o', markersize=5)
plt.show()

In [ ]:
# biorthogonality
j = 5
M0 = mra.refinement_matrix(j)
M0_t = mra_t.refinement_matrix(j)
assert np.allclose(M0.T @ M0_t, np.identity(M0.shape[1]))

# Dual MRA & Boundary Conditions & d > 2

In [ ]:
d = 3
d_t = 5
mra = PrimalMRA(d, bc=True)
mra_t = DualMRA(d, d_t, bc=True)

In [ ]:
j = 5
M0 = mra_t.refinement_matrix(j)
plt.rcParams['figure.figsize'] = [5, 10]
plt.spy(M0, marker='o', markersize=5)
plt.show()

In [ ]:
# biorthogonality
j = 5
M0 = mra.refinement_matrix(j)
M0_t = mra_t.refinement_matrix(j)
assert np.allclose(M0.T @ M0_t, np.identity(M0.shape[1]))

# Wavelet Basis

In [ ]:
d = 3
d_t = 5
wb = WaveletBasis(d, d_t)

In [ ]:
j = 5
M0, M1, M0_t, M1_t = wb.refinement_matrix(j, full=True)
assert np.allclose(M1, wb.refinement_matrix(j))

plt.rcParams['figure.figsize'] = [8.5, 17]
plt.subplot(221)
plt.spy(M0, marker='o', markersize=5)
plt.subplot(222)
plt.spy(M1, marker='o', markersize=5)
plt.subplot(223)
plt.spy(M0_t, marker='o', markersize=5)
plt.subplot(224)
plt.spy(M1_t, marker='o', markersize=5)
plt.show()

In [ ]:
# symmetry
assert np.allclose(M0, M0[::-1, ::-1])
assert np.allclose(M1, M1[::-1, ::-1])
assert np.allclose(M0_t, M0_t[::-1, ::-1])
assert np.allclose(M1_t, M1_t[::-1, ::-1])

In [ ]:
# biorthogonality
M = np.hstack((M0, M1))
M_t = np.hstack((M0_t, M1_t))
assert np.allclose(M.T @ M_t, np.identity(M.shape[0]))

In [ ]:
wb_test = WaveletBasis(3, 5)

GL_test = wb_test.GL
GL_ref = np.array([
    [3/4, 0, 0],
    [-12123/10240, -63/256, 35/256],
    [535021/614400, 147/5120, 21/1024],
    [-31139/204800, 5481/5120, -497/1024],
    [-42149/102400, -3969/2560, -7/512],
    [5659/61440, 85/512, 735/512],
    [9319/40960, 699/1024, -1295/1024],
    [-1103/204800, -3/5120, -117/1024],
    [-13273/153600, -311/1280, 253/768],
    [-797/51200, -57/1280, 17/256],
    [61/4096, 21/512, -25/512],
    [61/12288, 7/512, -25/1536]])
assert np.allclose(GL_test, GL_ref)

GL_t_test = wb_test.GL_t
GL_t_ref = np.array([
    [4/9, 0, 0],
    [-8/9, 0, 0],
    [2/3, -1/4, 0],
    [-2/9, 3/4, 0],
    [0, -3/4, -1/4],
    [0, 1/4, 3/4],
    [0, 0, -3/4],
    [0, 0, 1/4]])
assert np.allclose(GL_t_test, GL_t_ref)

- Primbs - 2006, Beispiel 5.9

In [ ]:
j = 4
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
wb.plot(j)

In [ ]:
j = 4
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
for k in range((d + d_t - 2) // 2):
    wb.plot(j, k, boundary=True)

- Primbs - 2006, Abbildung 5.4

In [ ]:
j = 4
G = wb.gramian(j)

bs = wb.basis_functions(j)
n = G.shape[0]
G_quad = np.empty((n, n))
for k in range(n):
    supp_k = wb.support(j, k)
    for m in range(n):
        supp_m = wb.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            G_quad[k, m] = 0.
        else:
            G_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(G, G_quad)

In [ ]:
# riesz constants
n = G.shape[0]
d1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
d2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("d1 =", d1)
print("d2 =", d2)
print("d2/d1 =", d2/d1)

tests = [(2, 2, 5), (2, 4, 5), (2, 6, 5), (2, 8, 5),
         (3, 3, 5), (3, 5, 5), (3, 7, 5), (3, 9, 6),
         (4, 6, 6), (4, 8, 6)]
results = []
refs = 1 / np.sqrt(2) * np.array([
    [0.832, 1.589], [0.821, 1.618], [0.817, 1.629], [0.817, 1.908],
    [0.507, 2.038], [0.489, 2.058], [0.512, 2.130], [0.516, 3.067],
    [0.329, 3.100], [0.330, 4.802]])
for d_test, d_t_test, j_test in tests:
    G_test = WaveletBasis(d_test, d_t_test).gramian(j_test)
    n = G_test.shape[0]
    d1_test = np.sqrt(eigvalsh(G_test, subset_by_index=[0, 0])[0])
    d2_test = np.sqrt(eigvalsh(G_test, subset_by_index=[n-1, n-1])[0])
    results.append([d1_test, d2_test])
assert np.allclose(results, refs, rtol=5e-2)

- Primbs - 2006, Tabelle 6.9

# Wavelet Basis & Boundary Conditions & d = 2

In [ ]:
d = 2
d_t = 4
wb = WaveletBasis(d, d_t, bc=True)

In [ ]:
j = 5
M0, M1, M0_t, M1_t = wb.refinement_matrix(j, full=True)
assert np.allclose(M1, wb.refinement_matrix(j))

plt.rcParams['figure.figsize'] = [8.5, 17]
plt.subplot(221)
plt.spy(M0, marker='o', markersize=5)
plt.subplot(222)
plt.spy(M1, marker='o', markersize=5)
plt.subplot(223)
plt.spy(M0_t, marker='o', markersize=5)
plt.subplot(224)
plt.spy(M1_t, marker='o', markersize=5)
plt.show()

In [ ]:
# symmetry
assert np.allclose(M0, M0[::-1, ::-1])
assert np.allclose(M1, M1[::-1, ::-1])
assert np.allclose(M0_t, M0_t[::-1, ::-1])
assert np.allclose(M1_t, M1_t[::-1, ::-1])

In [ ]:
# biorthogonality
M = np.hstack((M0, M1))
M_t = np.hstack((M0_t, M1_t))
assert np.allclose(M.T @ M_t, np.identity(M.shape[0]))

In [ ]:
j = 4
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
wb.plot(j)

In [ ]:
j = 4
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
for k in range(d_t // 2):
    wb.plot(j, k, boundary=True)

In [ ]:
j = 4
G = wb.gramian(j)

bs = wb.basis_functions(j)
n = G.shape[0]
G_quad = np.empty((n, n))
for k in range(n):
    supp_k = wb.support(j, k)
    for m in range(n):
        supp_m = wb.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            G_quad[k, m] = 0.
        else:
            G_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(G, G_quad)

In [ ]:
# riesz constants
n = G.shape[0]
d1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
d2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("d1 =", d1)
print("d2 =", d2)
print("d2/d1 =", d2/d1)

# Wavelet Basis & Boundary Conditions & d > 2

In [ ]:
d = 3
d_t = 5
wb = WaveletBasis(d, d_t, bc=True)

In [ ]:
j = 5
M0, M1, M0_t, M1_t = wb.refinement_matrix(j, full=True)
assert np.allclose(M1, wb.refinement_matrix(j))

plt.rcParams['figure.figsize'] = [8.5, 17]
plt.subplot(221)
plt.spy(M0, marker='o', markersize=5)
plt.subplot(222)
plt.spy(M1, marker='o', markersize=5)
plt.subplot(223)
plt.spy(M0_t, marker='o', markersize=5)
plt.subplot(224)
plt.spy(M1_t, marker='o', markersize=5)
plt.show()

In [ ]:
# symmetry
assert np.allclose(M0, M0[::-1, ::-1])
assert np.allclose(M1, M1[::-1, ::-1])
assert np.allclose(M0_t, M0_t[::-1, ::-1])
assert np.allclose(M1_t, M1_t[::-1, ::-1])

In [ ]:
# biorthogonality
M = np.hstack((M0, M1))
M_t = np.hstack((M0_t, M1_t))
assert np.allclose(M.T @ M_t, np.identity(M.shape[0]))

In [ ]:
j = 4
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
wb.plot(j)

In [ ]:
j = 4
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
for k in range((d + d_t - 2) // 2):
    wb.plot(j, k, boundary=True)

In [ ]:
j = 4
G = wb.gramian(j)

bs = wb.basis_functions(j)
n = G.shape[0]
G_quad = np.empty((n, n))
for k in range(n):
    supp_k = wb.support(j, k)
    for m in range(n):
        supp_m = wb.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            G_quad[k, m] = 0.
        else:
            G_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(G, G_quad)

In [ ]:
# riesz constants
n = G.shape[0]
d1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
d2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("d1 =", d1)
print("d2 =", d2)
print("d2/d1 =", d2/d1)

# Multi-Scale Wavelet Basis

In [ ]:
d = 3
d_t = 5
mwb = MultiscaleWaveletBasis(d, d_t)

In [ ]:
j0 = 4
s = 2
G = mwb.gramian(s, j0)

plt.rcParams['figure.figsize'] = [7, 7]
plt.spy(G, marker='o', markersize=1)
plt.show()

In [ ]:
# riesz constants
n = G.shape[0]
C1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
C2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("C1 =", C1)
print("C2 =", C2)
print("C2/C1 =", C2/C1)

tests = [(2, 2, 2), (2, 4, 3), (3, 3, 3), (3, 5, 4)]
results = []
refs = [[0.436, 1.047], [0.416, 1.141], [0.396, 1.195], [0.385, 1.209],
        [0.454, 1.112], [0.442, 1.145], [0.437, 1.152], [0.435, 1.156],
        [0.280, 1.373], [0.231, 1.450], [0.209, 1.507], [0.191, 1.518],
        [0.249, 1.441], [0.241, 1.456], [0.238, 1.466], [0.237, 1.470]]
for d_test, d_t_test, j0_test in tests:
    mwb_test = MultiscaleWaveletBasis(d_test, d_t_test)
    for s_test in [1, 2, 3, 4]:
        G_test = mwb_test.gramian(s_test, j0_test)
        n = G_test.shape[0]
        C1_test = np.sqrt(eigvalsh(G_test, subset_by_index=[0, 0])[0])
        C2_test = np.sqrt(eigvalsh(G_test, subset_by_index=[n-1, n-1])[0])
        results.append([C1_test, C2_test])
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2006, Tabelle 6.11

In [ ]:
# riesz constants
n = G.shape[0]
D = np.diag(1 / np.sqrt(np.diag(G)))
G = D @ G @ D
C1 = np.sqrt(eigvalsh(G, subset_by_index=[0, 0])[0])
C2 = np.sqrt(eigvalsh(G, subset_by_index=[n-1, n-1])[0])
print("C1 =", C1)
print("C2 =", C2)
print("C2/C1 =", C2/C1)

# diagonal scaling
tests = [(2, 2, 2), (2, 4, 3), (3, 3, 3), (3, 5, 4)]
results = []
refs = [[0.625, 1.236], [0.586, 1.332], [0.549, 1.384], [0.527, 1.396],
        [0.625, 1.331], [0.604, 1.362], [0.595, 1.370], [0.590, 1.374],
        [0.412, 1.531], [0.364, 1.611], [0.324, 1.648], [0.290, 1.657],
        [0.375, 1.675], [0.348, 1.692], [0.336, 1.704], [0.329, 1.714]]
for d_test, d_t_test, j0_test in tests:
    mwb_test = MultiscaleWaveletBasis(d_test, d_t_test)
    for s_test in [1, 2, 3, 4]:
        G_test = mwb_test.gramian(s_test, j0_test)
        D = np.diag(1 / np.sqrt(np.diag(G_test)))
        G_test = D @ G_test @ D
        n = G_test.shape[0]
        C1_test = np.sqrt(eigvalsh(G_test, subset_by_index=[0, 0])[0])
        C2_test = np.sqrt(eigvalsh(G_test, subset_by_index=[n-1, n-1])[0])
        results.append([C1_test, C2_test])
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2006, Tabelle 6.12

# Multi-Scale Wavelet Basis & Boundary Conditions & d = 2

In [ ]:
d = 2
d_t = 4
mwb = MultiscaleWaveletBasis(d, d_t, bc=True)

In [ ]:
j0 = 4
s = 2
nu = 1
A = mwb.inner_product(s, j0, nu)

plt.rcParams['figure.figsize'] = [7, 7]
plt.spy(A, marker='o', markersize=1, precision=1e-12)
plt.show()

In [ ]:
# uniform stability
D = np.diag(1 / np.sqrt(np.diag(A)))
A = D @ A @ D
cond = np.linalg.cond(A)
print("cond =", cond)

In [ ]:
# j0 according to dual boundary functions
tests = [(2, 2, 3), (2, 4, 4)]
results = []
refs = [25.274, 28.070, 30.337,
        103.087, 142.560, 154.237]
for d_test, d_t_test, j0_test in tests:
    mwb_test = MultiscaleWaveletBasis(d_test, d_t_test, bc=True)
    for s_test in [0, 1, 2]:
        A_test = mwb_test.inner_product(s_test, j0_test, nu=1)
        D = np.diag(1 / np.sqrt(np.diag(A_test)))
        A_test = D @ A_test @ D
        cond_test = np.linalg.cond(A_test)
        results.append(cond_test)
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2006, Tabelle 6.13

# Multi-Scale Wavelet Basis & Boundary Conditions & d > 2

In [ ]:
d = 3
d_t = 5
mwb = MultiscaleWaveletBasis(d, d_t, bc=True)

In [ ]:
j0 = 4
s = 2
nu = 1
A = mwb.inner_product(s, j0, nu)

plt.rcParams['figure.figsize'] = [7, 7]
plt.spy(A, marker='o', markersize=1, precision=1e-12)
plt.show()

- Primbs - 2006, Abbildung 6.1

In [ ]:
# uniform stability
D = np.diag(1 / np.sqrt(np.diag(A)))
A = D @ A @ D
cond = np.linalg.cond(A)
print("cond =", cond)

In [ ]:
# j0 according to dual boundary functions
tests = [(3, 3, 4), (3, 5, 5)]
results = []
refs = [38.885, 48.876, 49.232,
        155.579, 209.972, 215.234]
for d_test, d_t_test, j0_test in tests:
    mwb_test = MultiscaleWaveletBasis(d_test, d_t_test, bc=True)
    for s_test in [0, 1, 2]:
        A_test = mwb_test.inner_product(s_test, j0_test, nu=1)
        D = np.diag(1 / np.sqrt(np.diag(A_test)))
        A_test = D @ A_test @ D
        cond_test = np.linalg.cond(A_test)
        results.append(cond_test)
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2006, Tabelle 6.13

In [ ]:
# j0 according to boundary wavelets
tests = [(3, 3, 3), (3, 5, 4)]
results = []
refs = [9.855, 12.238, 12.582, 12.725, 12.816, 12.852,
        38.885, 52.969, 54.261, 54.877, 55.092, 55.185]
for d_test, d_t_test, j0_test in tests:
    mwb_test = MultiscaleWaveletBasis(d_test, d_t_test, bc=True)
    for s_test in [0, 1, 2, 3, 4, 5]:
        A_test = mwb_test.inner_product(s_test, j0_test, nu=1)
        D = np.diag(1 / np.sqrt(np.diag(A_test)))
        A_test = D @ A_test @ D
        cond_test = np.linalg.cond(A_test)
        results.append(cond_test)
assert np.allclose(results, refs, atol=1e-3)

- Primbs - 2006, Tabelle 6.14